In [52]:
from pyspark.sql import SparkSession
from deep_translator import GoogleTranslator

spark = (
    
    SparkSession
            .builder
            .config('spark.jars.packages', 'io.delta:delta-core_2.12:2.1.0')
            .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
            .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')
            .getOrCreate()
            
)

In [53]:
lotr = (
    spark
    .read
    .format('delta')
    .load('../datalake/bronze/lotr/')
)

In [54]:
def translate_en_to_pt(text: str):
    
    return GoogleTranslator(source='english', target='portuguese').translate(text)

In [55]:
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [56]:
translated = F.udf(translate_en_to_pt, T.StringType())

In [57]:
lotr = lotr.withColumn('dialog', translated('dialog'))

In [58]:
lotr = lotr.withColumnRenamed('dialog', 'dialogos')
lotr = lotr.withColumnRenamed('movie', 'filme')
lotr = lotr.withColumnRenamed('char', 'personagem')

In [59]:
lotr.show(n=1, vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------------------
 Unnamed:0  | 0                                                             
 personagem | DEAGOL                                                        
 dialogos   | Oh Smeagol Eu tenho um! , Eu tenho um peixe Smeagol, Smeagol! 
 filme      | The Return of the King                                        
only showing top 1 row



In [63]:
(
    lotr
    .write
    .format('delta')
    .mode('overwrite')
    .save('../datalake/silver/lotr_pt_br')

)

In [ ]:
lotr.show()